In [26]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from statistics import multimode
from statistics import mode
import itertools
from openpyxl import load_workbook

""" 
    This file contains all the helper functions needed to calculate the burst variables 
    and inter-reward interval variables.
"""

def cleanup(filepath):
    """ This function is used to cleanup the excel output file for further analysis

    Args:
        filepath (_str_): the filename of the raw excel output file

    Returns:
        _pd.DataFrame_: a cleaned dataframe
    """
    #print(filepath)
    # import data and transpose
    df_raw = pd.read_excel(filepath,header=None).T

    # modify the header
    new_header = df_raw.iloc[0]   #grab the first row for the header
    df = df_raw[1:]               #take the data except the header row
    df.columns = new_header 
    df.reset_index(drop=True, inplace=True)

    # add new column for visualization 
    df['Start Datetime'] = pd.to_datetime(df['Start Date'] + ' ' + df['Start Time'])
    df['End Datetime'] = pd.to_datetime(df['End Date'] + ' ' + df['End Time'])
    df['Timeout'] = (df['Active Lever Presses'] - df['Reward']).astype('int32')

    # drop unnecessary columns
    df.drop(['Start Date','Start Time','End Date','End Time'], axis=1, inplace=True)

    # change data types
    cols = df.columns.tolist()
    for col in cols[:-1]:
        name = col.lower()
        if ('active' in name) or ('reward' in name) or ('timeout' in name):
            df[col] = df[col].astype('int32')
            df[col] = df[col].replace(0,np.nan)
        else:
            pass
        
    # drop columns which all values are Nan
    df.dropna(how='all', axis=1, inplace=True)

    # reorder the columns
    new_columns = df.columns.tolist()
    new_columns.insert(0, new_columns.pop(new_columns.index('Subject')))
    new_columns.insert(1, new_columns.pop(new_columns.index('Start Datetime')))
    new_columns.insert(2, new_columns.pop(new_columns.index('End Datetime')))

    #
    idx = new_columns.index('Reward') + 1
    new_columns.insert(idx, new_columns.pop(new_columns.index('Timeout')))

    # fill the nan with 0
    df = df[new_columns]
    df.fillna(0,inplace=True)
    #print('CLEANING COMPLETED')
    # output the result dataframe
    return df


def get_mode(lst):
    """ This function returns the mode of the input list

    Args:
        lst (_type_): list of int

    Returns:
        _type_: int
    """
    if len(lst) == 0:
        return np.NaN
    return mode(lst)


def get_bursts(lst, duration = 120):
    """ This function retrieves the "bursts" (cluster of rewards happened within 2 mins) from rewards

    Args:
        lst (_type_): list of int

    Returns:
        _type_: 2d list
    """
    interval = duration
    allBursts = []

    i = 0
    while i < len(lst):
        oneBurst = []
        limit = lst[i] + interval
        j = i
        while j < len(lst) and lst[j] <= limit:
            oneBurst.append(lst[j])
            j += 1
        allBursts.append(oneBurst)
        i = j
        
    return allBursts


def get_mean_num_rewards(lst):
    """ This function returns the mean number of rewards out of all the bursts

    Args:
        lst (_type_): list of int

    Returns:
        _type_: int
    """
    bursts = [i for i in lst if len(i) > 1]
    total_rewards = len(list(itertools.chain.from_iterable(bursts)))
    total_bursts = len(bursts)
    
    if total_bursts == 0: 
        return 0
    
    return round(total_rewards / total_bursts,2)


def get_burst_rewards_pct(lst):
    """ This function returns the percentage of rewards that fall in burst out of all the rewards

    Args:
        lst (_type_): 2d list

    Returns:
        _type_: int
    """
    numRewards = len(list(itertools.chain.from_iterable(lst)))
    if numRewards == 0:
        return np.NaN
    bursts = [i for i in lst if len(i) > 1]
    numBurstRewards = len(list(itertools.chain.from_iterable(bursts)))
    return round(numBurstRewards/numRewards,2) * 100 


def get_max_burst(lst):
    """ This function returns the max number of rewards amoung all the bursts

    Args:
        lst (_type_): 2d list 

    Returns:
        _type_: int
    """
    bursts = [i for i in lst if len(i) > 1]
    if len(bursts) == 0:
        return 0
    return max([len(i) for i in bursts])


def calculations_single(df):
    """ This function returns the dataframe of all the calculated variables (original version)

    Args:
        df (_type_): pd.DataFrame, cleaned dataframe processed in the cleanup function

    Returns:
        _type_: pd.DataFrame
    """
    # combine all reward timestamp into a column of lists
    filtered_cols = ['Subject'] + [col for col in df.columns if 'Reward ' in col]
    df_reward = df[filtered_cols][df.Subject!=0].sort_values('Subject').reset_index().drop('index',axis=1)
    df_reward['allRewards'] = df_reward.iloc[:,1:].values.tolist()

    # get the filtered df
    dff = df_reward[['Subject', 'allRewards']]

    # retrieve the inter-reward intervals, filtering out negatives
    dff['Intervals'] = dff['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
    dff['cleanedIntervals'] = dff['Intervals'].apply(lambda lst: [val for val in lst if val > 0])

    # calculate needed traits related to the inter-reward intervals
    dff['meanInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.mean(x), 2))
    dff['stdInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.std(x), 2))
    dff['modeInterval'] = dff['cleanedIntervals'].apply(get_mode)

    # filtering the rewards (zero values)
    dff['cleanedRewards'] = dff['allRewards'].apply(lambda lst: [val for val in lst if val > 0])
    dff['totalRewards'] = dff['cleanedRewards'].apply(lambda x: len(x))

    # retrieve the "bursts" (cluster of rewards happened within 2 mins) from rewards 
    dff['rawBurst'] = dff['cleanedRewards'].apply(get_bursts)
    dff['numBurst'] = dff['rawBurst'].apply(lambda x: len([i for i in x if len(i)>1]))

    # get the mean number of rewards across all the bursts
    dff['meanNumRewards'] = dff['rawBurst'].apply(get_mean_num_rewards)

    # get the percentage of rewards that fall in burst out of all the rewards
    dff['pctRewards'] = dff['rawBurst'].apply(get_burst_rewards_pct)

    # get the maximum number of rewards contain in a single burst in one session
    dff['maxBurst'] = dff['rawBurst'].apply(get_max_burst)

    # select needed columns for output df
    output_cols = ['Subject', 'meanInterval', 'stdInterval', 'modeInterval','meanNumRewards', 'numBurst', 'maxBurst', 'pctRewards']
    dff_out = dff[output_cols]
    
    return dff_out


def get_sheetnames_xlsx(file_name):
    """This function returns the worksheet names from an given Excel workbook

    Args:
        file_name (_type_): str

    Returns:
        _type_: list of strings
    """
    wb = load_workbook(file_name, read_only=True, keep_links=False)
    return wb.sheetnames


def one_calculation(file_name, sheet_name, rats):
    """ This function returns the dataframe of all the calculated variables (VK version)

    Args:
        file_name (_str_): the name of the excel workbook
        sheet_name (_str_): 1 sheetname from all sheetnames within the chosen workbook
        rats (_str_): all the rats in the chosen cohort 

    Returns:
        _type_: pd.DataFrame
    """
    # read excel sheet
    df = pd.read_excel(file_name, sheet_name=sheet_name).T
    
    # reset header
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header

    # filter rows and columns
    rats = list(set(rats) & set(list(df.index))) # get the intersection
    df = df.loc[rats]
    col_reward = [col for col in df.columns if 'V' in col] # V for reward timestamp
    df = df[col_reward]

    # drop zeros
    df.replace(0, np.NaN, inplace=True)
    df.dropna(how='all', axis=1, inplace=True)
    df.replace(np.NaN, 0, inplace=True)
    df.reset_index(inplace=True)
    
    # group all rewards
    df['allRewards'] = df.iloc[:,1:].values.tolist()
    
    # get the filtered df
    dff = df[['index','allRewards']]
    
    # retrieve the inter-reward intervals, filtering out negatives
    dff['Intervals'] = dff['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
    dff['cleanedIntervals'] = dff['Intervals'].apply(lambda lst: [val for val in lst if val > 0])
    
    # calculate needed traits related to the inter-reward intervals
    dff['meanInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.mean(x), 2))
    dff['stdInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.std(x), 2))
    dff['modeInterval'] = dff['cleanedIntervals'].apply(get_mode)
    
    # filtering the rewards (zero values)
    dff['cleanedRewards'] = dff['allRewards'].apply(lambda lst: [val for val in lst if val > 0])
    dff['totalRewards'] = dff['cleanedRewards'].apply(lambda x: len(x))
    
    # retrieve the "bursts" (cluster of rewards happened within 2 mins) from rewards 
    dff['rawBurst'] = dff['cleanedRewards'].apply(get_bursts)
    dff['numBurst'] = dff['rawBurst'].apply(lambda x: len([i for i in x if len(i)>1]))
    
    # get the mean number of rewards across all the bursts
    dff['meanNumRewards'] = dff['rawBurst'].apply(get_mean_num_rewards)
    # get the percentage of rewards that fall in burst out of all the rewards
    dff['pctRewards'] = dff['rawBurst'].apply(get_burst_rewards_pct)
    # get the maximum number of rewards contain in a single burst in one session
    dff['maxBurst'] = dff['rawBurst'].apply(get_max_burst)
    
    # select needed columns for output df
    output_cols = ['index', 'meanInterval', 'stdInterval', 'modeInterval','meanNumRewards', 'numBurst', 'maxBurst', 'pctRewards']
    dff_out = dff[output_cols]
    
    # rename one column
    dff_out.rename(columns={"index": "subject"},inplace=True)
    
    # add trial id
    trial_id = re.findall(r'((?:SHA|LGA)[0-9]+)',sheet_name.upper())[0]
    dff_out['trial_id'] = [trial_id] * len(dff_out)
    
    return dff_out

In [10]:
filepath = '/Users/yunyihuang/Desktop/gl_data/COCAINE/LGA/BSB273BC08HSLGA01_output.xlsx'

In [11]:
df = cleanup(filepath)
df.head()

,Subject,Start Datetime,End Datetime,Filename,Experiment,Group,Box,MSN,FR,Active Lever Presses,...,Timeout Press 53,Timeout Press 54,Timeout Press 55,Timeout Press 56,Timeout Press 57,Timeout Press 58,Timeout Press 59,Timeout Press 60,Timeout Press 61,Timeout Press 62
0,F801,2019-07-11 09:16:44,2019-07-11 15:42:38,C:\MED-PC IV\DATA\C08HSLGA01,0,0,1,GWAS 6h,0,113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,F802,2019-07-11 09:16:49,2019-07-11 15:42:38,C08HSLGA01\n,0,0,2,GWAS 6h,0,101.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F803,2019-07-11 09:16:52,2019-07-11 15:42:38,C08HSLGA01\n,0,0,3,GWAS 6h,0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,F804,2019-07-11 09:16:54,2019-07-11 15:42:38,C08HSLGA01\n,0,0,4,GWAS 6h,0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,F805,2019-07-11 09:16:58,2019-07-11 15:42:38,C08HSLGA01\n,0,0,5,GWAS 6h,0,144.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df.shape

(16, 493)

In [13]:
df.columns.tolist()

['Subject',
 'Start Datetime',
 'End Datetime',
 'Filename',
 'Experiment',
 'Group',
 'Box',
 'MSN',
 'FR',
 'Active Lever Presses',
 'Inactive Lever Presses',
 'Reward',
 'Timeout',
 'Active 1',
 'Active 2',
 'Active 3',
 'Active 4',
 'Active 5',
 'Active 6',
 'Active 7',
 'Active 8',
 'Active 9',
 'Active 10',
 'Active 11',
 'Active 12',
 'Active 13',
 'Active 14',
 'Active 15',
 'Active 16',
 'Active 17',
 'Active 18',
 'Active 19',
 'Active 20',
 'Active 21',
 'Active 22',
 'Active 23',
 'Active 24',
 'Active 25',
 'Active 26',
 'Active 27',
 'Active 28',
 'Active 29',
 'Active 30',
 'Active 31',
 'Active 32',
 'Active 33',
 'Active 34',
 'Active 35',
 'Active 36',
 'Active 37',
 'Active 38',
 'Active 39',
 'Active 40',
 'Active 41',
 'Active 42',
 'Active 43',
 'Active 44',
 'Active 45',
 'Active 46',
 'Active 47',
 'Active 48',
 'Active 49',
 'Active 50',
 'Active 51',
 'Active 52',
 'Active 53',
 'Active 54',
 'Active 55',
 'Active 56',
 'Active 57',
 'Active 58',
 'Active 59',

In [15]:
filtered_cols = ['Subject'] + [col for col in df.columns if 'Reward ' in col]
df_reward = df[filtered_cols][df.Subject!=0].sort_values('Subject').reset_index().drop('index',axis=1)
df_reward['allRewards'] = df_reward.iloc[:,1:].values.tolist()

dff = df_reward[['Subject', 'allRewards']]
dff.head()

,Subject,allRewards
0,F801,"[2309.0, 2349.0, 2410.0, 2446.0, 2526.0, 2546...."
1,F802,"[307.0, 422.0, 2693.0, 2925.0, 3242.0, 3472.0,..."
2,F803,"[2788.0, 3120.0, 4917.0, 7356.0, 16818.0, 0.0,..."
3,F804,"[4551.0, 4723.0, 6709.0, 6859.0, 8197.0, 8241...."
4,F805,"[100.0, 166.0, 188.0, 213.0, 492.0, 516.0, 628..."


In [17]:
dff.allRewards.values[0]

[2309.0,
 2349.0,
 2410.0,
 2446.0,
 2526.0,
 2546.0,
 2922.0,
 3128.0,
 3325.0,
 3538.0,
 3642.0,
 4011.0,
 4144.0,
 4509.0,
 4837.0,
 5147.0,
 5492.0,
 5846.0,
 6116.0,
 6554.0,
 6819.0,
 7070.0,
 7258.0,
 7465.0,
 7811.0,
 8135.0,
 8296.0,
 8566.0,
 8867.0,
 9309.0,
 9622.0,
 9808.0,
 10040.0,
 10325.0,
 10593.0,
 10717.0,
 10972.0,
 11137.0,
 11392.0,
 11556.0,
 11739.0,
 11922.0,
 12098.0,
 12200.0,
 12438.0,
 12665.0,
 12965.0,
 13167.0,
 13354.0,
 13564.0,
 13769.0,
 14026.0,
 14182.0,
 14555.0,
 14982.0,
 15289.0,
 15512.0,
 15644.0,
 15863.0,
 16065.0,
 16242.0,
 16425.0,
 16536.0,
 16777.0,
 16939.0,
 17213.0,
 17484.0,
 17640.0,
 17867.0,
 18026.0,
 18266.0,
 18500.0,
 18746.0,
 18958.0,
 19176.0,
 19357.0,
 19452.0,
 19646.0,
 19746.0,
 19924.0,
 20155.0,
 20402.0,
 20539.0,
 20714.0,
 20887.0,
 21447.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [18]:
dff['Intervals'] = dff['allRewards'].apply(lambda lst:[j-i for i, j in zip(lst[:-1], lst[1:])])
dff['cleanedIntervals'] = dff['Intervals'].apply(lambda lst: [val for val in lst if val > 0])

dff.Intervals.values[0]

[40.0,
 61.0,
 36.0,
 80.0,
 20.0,
 376.0,
 206.0,
 197.0,
 213.0,
 104.0,
 369.0,
 133.0,
 365.0,
 328.0,
 310.0,
 345.0,
 354.0,
 270.0,
 438.0,
 265.0,
 251.0,
 188.0,
 207.0,
 346.0,
 324.0,
 161.0,
 270.0,
 301.0,
 442.0,
 313.0,
 186.0,
 232.0,
 285.0,
 268.0,
 124.0,
 255.0,
 165.0,
 255.0,
 164.0,
 183.0,
 183.0,
 176.0,
 102.0,
 238.0,
 227.0,
 300.0,
 202.0,
 187.0,
 210.0,
 205.0,
 257.0,
 156.0,
 373.0,
 427.0,
 307.0,
 223.0,
 132.0,
 219.0,
 202.0,
 177.0,
 183.0,
 111.0,
 241.0,
 162.0,
 274.0,
 271.0,
 156.0,
 227.0,
 159.0,
 240.0,
 234.0,
 246.0,
 212.0,
 218.0,
 181.0,
 95.0,
 194.0,
 100.0,
 178.0,
 231.0,
 247.0,
 137.0,
 175.0,
 173.0,
 560.0,
 -21447.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.

In [19]:
dff.cleanedIntervals.values[0]

[40.0,
 61.0,
 36.0,
 80.0,
 20.0,
 376.0,
 206.0,
 197.0,
 213.0,
 104.0,
 369.0,
 133.0,
 365.0,
 328.0,
 310.0,
 345.0,
 354.0,
 270.0,
 438.0,
 265.0,
 251.0,
 188.0,
 207.0,
 346.0,
 324.0,
 161.0,
 270.0,
 301.0,
 442.0,
 313.0,
 186.0,
 232.0,
 285.0,
 268.0,
 124.0,
 255.0,
 165.0,
 255.0,
 164.0,
 183.0,
 183.0,
 176.0,
 102.0,
 238.0,
 227.0,
 300.0,
 202.0,
 187.0,
 210.0,
 205.0,
 257.0,
 156.0,
 373.0,
 427.0,
 307.0,
 223.0,
 132.0,
 219.0,
 202.0,
 177.0,
 183.0,
 111.0,
 241.0,
 162.0,
 274.0,
 271.0,
 156.0,
 227.0,
 159.0,
 240.0,
 234.0,
 246.0,
 212.0,
 218.0,
 181.0,
 95.0,
 194.0,
 100.0,
 178.0,
 231.0,
 247.0,
 137.0,
 175.0,
 173.0,
 560.0]

In [20]:
# calculate needed traits related to the inter-reward intervals
dff['meanInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.mean(x), 2))
dff['stdInterval'] = dff['cleanedIntervals'].apply(lambda x: round(np.std(x), 2))
dff['modeInterval'] = dff['cleanedIntervals'].apply(get_mode)

# filtering the rewards (zero values)
dff['cleanedRewards'] = dff['allRewards'].apply(lambda lst: [val for val in lst if val > 0])
dff['totalRewards'] = dff['cleanedRewards'].apply(lambda x: len(x))

In [22]:
dff.head()

,Subject,allRewards,Intervals,cleanedIntervals,meanInterval,stdInterval,modeInterval,cleanedRewards,totalRewards
0,F801,"[2309.0, 2349.0, 2410.0, 2446.0, 2526.0, 2546....","[40.0, 61.0, 36.0, 80.0, 20.0, 376.0, 206.0, 1...","[40.0, 61.0, 36.0, 80.0, 20.0, 376.0, 206.0, 1...",225.15,95.69,183.0,"[2309.0, 2349.0, 2410.0, 2446.0, 2526.0, 2546....",86
1,F802,"[307.0, 422.0, 2693.0, 2925.0, 3242.0, 3472.0,...","[115.0, 2271.0, 232.0, 317.0, 230.0, 315.0, 75...","[115.0, 2271.0, 232.0, 317.0, 230.0, 315.0, 75...",315.51,271.37,197.0,"[307.0, 422.0, 2693.0, 2925.0, 3242.0, 3472.0,...",68
2,F803,"[2788.0, 3120.0, 4917.0, 7356.0, 16818.0, 0.0,...","[332.0, 1797.0, 2439.0, 9462.0, -16818.0, 0.0,...","[332.0, 1797.0, 2439.0, 9462.0]",3507.50,3521.63,332.0,"[2788.0, 3120.0, 4917.0, 7356.0, 16818.0]",5
3,F804,"[4551.0, 4723.0, 6709.0, 6859.0, 8197.0, 8241....","[172.0, 1986.0, 150.0, 1338.0, 44.0, 560.0, 43...","[172.0, 1986.0, 150.0, 1338.0, 44.0, 560.0, 43...",1057.20,1189.17,172.0,"[4551.0, 4723.0, 6709.0, 6859.0, 8197.0, 8241....",16
4,F805,"[100.0, 166.0, 188.0, 213.0, 492.0, 516.0, 628...","[66.0, 22.0, 25.0, 279.0, 24.0, 112.0, 60.0, 1...","[66.0, 22.0, 25.0, 279.0, 24.0, 112.0, 60.0, 1...",167.90,116.26,247.0,"[100.0, 166.0, 188.0, 213.0, 492.0, 516.0, 628...",127


In [23]:
dff.cleanedRewards.values[0]

[2309.0,
 2349.0,
 2410.0,
 2446.0,
 2526.0,
 2546.0,
 2922.0,
 3128.0,
 3325.0,
 3538.0,
 3642.0,
 4011.0,
 4144.0,
 4509.0,
 4837.0,
 5147.0,
 5492.0,
 5846.0,
 6116.0,
 6554.0,
 6819.0,
 7070.0,
 7258.0,
 7465.0,
 7811.0,
 8135.0,
 8296.0,
 8566.0,
 8867.0,
 9309.0,
 9622.0,
 9808.0,
 10040.0,
 10325.0,
 10593.0,
 10717.0,
 10972.0,
 11137.0,
 11392.0,
 11556.0,
 11739.0,
 11922.0,
 12098.0,
 12200.0,
 12438.0,
 12665.0,
 12965.0,
 13167.0,
 13354.0,
 13564.0,
 13769.0,
 14026.0,
 14182.0,
 14555.0,
 14982.0,
 15289.0,
 15512.0,
 15644.0,
 15863.0,
 16065.0,
 16242.0,
 16425.0,
 16536.0,
 16777.0,
 16939.0,
 17213.0,
 17484.0,
 17640.0,
 17867.0,
 18026.0,
 18266.0,
 18500.0,
 18746.0,
 18958.0,
 19176.0,
 19357.0,
 19452.0,
 19646.0,
 19746.0,
 19924.0,
 20155.0,
 20402.0,
 20539.0,
 20714.0,
 20887.0,
 21447.0]

In [31]:
dff['rawBurst'] = dff['cleanedRewards'].apply(lambda x: get_bursts(x,duration=180))

In [32]:
dff.rawBurst.values[0]

[[2309.0, 2349.0, 2410.0, 2446.0],
 [2526.0, 2546.0],
 [2922.0],
 [3128.0],
 [3325.0],
 [3538.0, 3642.0],
 [4011.0, 4144.0],
 [4509.0],
 [4837.0],
 [5147.0],
 [5492.0],
 [5846.0],
 [6116.0],
 [6554.0],
 [6819.0],
 [7070.0],
 [7258.0],
 [7465.0],
 [7811.0],
 [8135.0, 8296.0],
 [8566.0],
 [8867.0],
 [9309.0],
 [9622.0],
 [9808.0],
 [10040.0],
 [10325.0],
 [10593.0, 10717.0],
 [10972.0, 11137.0],
 [11392.0, 11556.0],
 [11739.0],
 [11922.0, 12098.0],
 [12200.0],
 [12438.0],
 [12665.0],
 [12965.0],
 [13167.0],
 [13354.0],
 [13564.0],
 [13769.0],
 [14026.0, 14182.0],
 [14555.0],
 [14982.0],
 [15289.0],
 [15512.0, 15644.0],
 [15863.0],
 [16065.0, 16242.0],
 [16425.0, 16536.0],
 [16777.0, 16939.0],
 [17213.0],
 [17484.0, 17640.0],
 [17867.0, 18026.0],
 [18266.0],
 [18500.0],
 [18746.0],
 [18958.0],
 [19176.0],
 [19357.0, 19452.0],
 [19646.0, 19746.0],
 [19924.0],
 [20155.0],
 [20402.0, 20539.0],
 [20714.0, 20887.0],
 [21447.0]]